<a href="https://colab.research.google.com/github/gulabpatel/Table_Detection/blob/main/00%3A%20pdfplumber_Invoice_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pdfplumber

In [8]:
import re

import requests
import pdfplumber
import pandas as pd
from collections import namedtuple

In [9]:
Inv = namedtuple('Inv', 'vend_num vend_name inv_dt due_dt inv_amt net_amt description')

In [10]:
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [11]:
ap_url = 'https://www.tabs3.com/support/sample/apreports.pdf'

In [12]:
ap = download_file(ap_url)

In [14]:
with pdfplumber.open(ap) as pdf:
    page = pdf.pages[9]
    text = page.extract_text()

In [15]:
print(text)

AccountsPayableReportPack
Summary Vendor List
Date: 08/23/2022 Summary Numerical Vendor List Page: 1
Jensen, Martin & Anderson, P.C.
Due Discount
Vendor Name Phone # Days % Days
(200) United Parcel Service
201 Fed Ex 800.622.1147
202 Software Technology, LLC 402-419-2200 5
203 Clerk of the County Court
204 Clerk of the District Court
205 Business Week
225 * Jackson/Wylinda
240 NELCO 800-266-4669 30 2.00 10
242 Office Supply Megastore 30 1.50 5
250 * D & B Real Estate Management Company 402-222-5543
400 Sprint Local & Long Distance 20
700 * Clean All Janitorial Services 30
999M Miscellaneous Vendor
Menu Reports|Vendors|VendorList
ThereportshownaboveisasummaryVendorList.TheprogramtabsforthisreportincludethestandardVendortabaswellastheOptionstab.TheOptionstaballowsyoutoselectwhetherthereportshouldbeprintedindetailorsummaryformat.Thelistshownaboveisprintedinsummaryformatinnumericalvendororder.
Definitions for Summary Vendor List
Date TheReportDateenteredwhenthelistwasrun.
Vendor Vendornumb

In [18]:
new_vend_re = re.compile(r'^\d{3}|\(\d{3}\) [A-Z].*')

In [19]:
for line in text.split('\n'):
    if new_vend_re.match(line):
        print(line)

(200) United Parcel Service
201 Fed Ex 800.622.1147
202 Software Technology, LLC 402-419-2200 5
203 Clerk of the County Court
204 Clerk of the District Court
205 Business Week
225 * Jackson/Wylinda
240 NELCO 800-266-4669 30 2.00 10
242 Office Supply Megastore 30 1.50 5
250 * D & B Real Estate Management Company 402-222-5543
400 Sprint Local & Long Distance 20
700 * Clean All Janitorial Services 30
999M Miscellaneous Vendor


In [20]:
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)
print(vend_num)
print(vend_name)

999M
Miscellaneous Vendor


In [26]:
with pdfplumber.open(ap) as pdf:
    page = pdf.pages[16]
    text = page.extract_text()

In [27]:
inv_line_re = re.compile(r'(\d{6}) (\d{6}) ([\d,]+\.\d{2}) [\sP]*([\d,]+\.\d{2}) [YN ]*\d (.*?) [*\s\d]')

In [28]:
line_items = []
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)    
    
    line = inv_line_re.search(line)
    if line:
        inv_dt = line.group(1)
        due_dt = line.group(2)
        inv_amt = line.group(3)
        net_amt = line.group(4)
        desc = line.group(5)
        line_items.append(Inv(vend_num, vend_name, inv_dt, due_dt, inv_amt, net_amt, desc))

In [ ]:
line_items

In [31]:
df = pd.DataFrame(line_items)

In [33]:
df

,vend_num,vend_name,inv_dt,due_dt,inv_amt,net_amt,description
0,200.01MLJ,3243387 032422 062422 75.00 P 75.00 1 Overnigh...,032422,062422,75.00,75.00,Overnight document delivery
1,3243387,032522 062522 75.00 P 75.00 3 Postage * 1,032522,062522,75.00,75.00,Postage
2,3243387,072422 072422 75.00 P 75.00 2 Overnight docume...,072422,072422,75.00,75.00,Overnight document delivery
3,312205,081822 082322 490.00 P 490.00 1 Software Maint...,081822,082322,490.00,490.00,Software Maintenance Agreement Tabs3
4,312206,081822 082322 239.00 P 239.00 2 Software Maint...,081822,082322,239.00,239.00,Software Maintenance Agreement Tabs3 General L...
5,312207,081822 082322 239.00 P 239.00 3 Software Maint...,081822,082322,239.00,239.00,Software Maintenance Agreement Tabs3 Accounts ...
6,312208,081822 082322 239.00 P 239.00 4 Software Maint...,081822,082322,239.00,239.00,Software Maintenance Agreement Tabs3 Trust Acc...
7,312209,081822 082322 800.00 P 800.00 5 Software Maint...,081822,082322,800.00,800.00,Software Maintenance Agreement PracticeMaster ...
8,312210,081822 082322 105.00 P 105.00 6 Software Maint...,081822,082322,105.00,105.00,Software Maintenance Agreement Tabs3 Taskbill
9,320559,082322 082822 205.00 205.00 7 Purchase Tabs3 D...,082322,082822,205.00,205.00,Purchase Tabs3 Device Interface Program


In [34]:
df['inv_dt'] = pd.to_datetime(df['inv_dt'])
df['due_dt'] = pd.to_datetime(df['due_dt'])

In [35]:
df

,vend_num,vend_name,inv_dt,due_dt,inv_amt,net_amt,description
0,200.01MLJ,3243387 032422 062422 75.00 P 75.00 1 Overnigh...,2022-03-24,2022-06-24,75.00,75.00,Overnight document delivery
1,3243387,032522 062522 75.00 P 75.00 3 Postage * 1,2022-03-25,2022-06-25,75.00,75.00,Postage
2,3243387,072422 072422 75.00 P 75.00 2 Overnight docume...,2022-07-24,2022-07-24,75.00,75.00,Overnight document delivery
3,312205,081822 082322 490.00 P 490.00 1 Software Maint...,2022-08-18,2022-08-23,490.00,490.00,Software Maintenance Agreement Tabs3
4,312206,081822 082322 239.00 P 239.00 2 Software Maint...,2022-08-18,2022-08-23,239.00,239.00,Software Maintenance Agreement Tabs3 General L...
5,312207,081822 082322 239.00 P 239.00 3 Software Maint...,2022-08-18,2022-08-23,239.00,239.00,Software Maintenance Agreement Tabs3 Accounts ...
6,312208,081822 082322 239.00 P 239.00 4 Software Maint...,2022-08-18,2022-08-23,239.00,239.00,Software Maintenance Agreement Tabs3 Trust Acc...
7,312209,081822 082322 800.00 P 800.00 5 Software Maint...,2022-08-18,2022-08-23,800.00,800.00,Software Maintenance Agreement PracticeMaster ...
8,312210,081822 082322 105.00 P 105.00 6 Software Maint...,2022-08-18,2022-08-23,105.00,105.00,Software Maintenance Agreement Tabs3 Taskbill
9,320559,082322 082822 205.00 205.00 7 Purchase Tabs3 D...,2022-08-23,2022-08-28,205.00,205.00,Purchase Tabs3 Device Interface Program


In [36]:
df['inv_amt'] = df['inv_amt'].map(lambda x: float(x.replace(',', '')))

In [37]:
df['net_amt'] = df['net_amt'].map(lambda x: float(x.replace(',', '')))

In [38]:
df.sum()

<ipython-input-38-7e5fdb616c56>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.sum()


vend_num       200.01MLJ3243387324338731220531220631220731220...
vend_name      3243387 032422 062422 75.00 P 75.00 1 Overnigh...
inv_amt                                                 22476.31
net_amt                                                 22476.31
description    Overnight document deliveryPostageOvernight do...
dtype: object

In [ ]:
df.to_csv('inv_lines.csv')